In [1]:
#created in Google Colaboratory by NIkolas Rakryan Widagdo

Answer to question 1

In [2]:
import sqlite3 as sql
from sqlite3 import Error

In [3]:
con=sql.connect('database.db')
cur=con.cursor()

In [4]:
def sql_connection():
  try:
    con=sql.connect('database.db')
    return con
  except Error:
    print(Error)

def sql_table(con):
  cur.execute("""CREATE TABLE IF NOT EXISTS transactions
              (tx_id integer,id integer, insert_time text,
              tx_amount real, tx_type text, status text)""")
  con.commit()

con=sql_connection()
sql_table(con)

In [5]:
def sql_insert(con,entities):
  cur=con.cursor()
  cur.executemany('INSERT INTO transactions(tx_id ,id, insert_time, tx_amount, tx_type, status) VALUES(?,?,?,?,?,?)',entities)
  con.commit()

entities=[(10,1,"2020-06-20 09:10",50,"buy","open"),
          (11,2,"2020-06-20 09:22",57,"sell","close"),
          (12,6,"2020-06-20 09:30",100,"buy","close"),
          (13,3,"2020-06-20 09:42",6,"buy","open"),
          (14,5,"2020-06-20 09:45",23,"sell","close"),
          (15,2,"2020-06-20 10:30",4,"buy","close"),
          (16,1,"2020-06-20 10:33",55,"sell","open"),
          (17,4,"2020-06-20 10:55",765,"buy","open"),
          (18,3,"2020-06-20 11:30",6,"sell","open"),
          (19,1,"2020-06-20 11:40",34,"sell","open")]

sql_insert(con,entities)

In [6]:
def sql_fetch_latest(con):
  cur=con.cursor()
  cur.execute("""SELECT * FROM transactions 
              GROUP BY id 
              HAVING insert_time=max(insert_time) 
              ORDER BY insert_time""")
  rows=cur.fetchall()
  for row in rows:
    print(row)
sql_fetch_latest(con)

(12, 6, '2020-06-20 09:30', 100.0, 'buy', 'close')
(14, 5, '2020-06-20 09:45', 23.0, 'sell', 'close')
(15, 2, '2020-06-20 10:30', 4.0, 'buy', 'close')
(17, 4, '2020-06-20 10:55', 765.0, 'buy', 'open')
(18, 3, '2020-06-20 11:30', 6.0, 'sell', 'open')
(19, 1, '2020-06-20 11:40', 34.0, 'sell', 'open')


Answer to question 2

In [7]:
def sql_connection2():
  try:
    con=sql.connect('database.db')
    return con
  except Error:
    print(Error)

def sql_table2(con):
  cur.execute("""CREATE TABLE IF NOT EXISTS salesdata
              (date text,sales integer)""")
  con.commit()

con=sql_connection()
sql_table2(con)

In [8]:
def sql_insert2(con,entities):
  cur=con.cursor()
  cur.executemany('INSERT INTO salesdata(date, sales) VALUES(?,?)',entities)
  con.commit()

entities=[("2021-01-01",100),
          ("2021-01-02",34),
          ("2021-01-02",123),
          ("2021-01-02",134),
          ("2021-01-03",145),
          ("2021-01-03",24),
          ("2021-01-04",541),
          ("2021-01-04",636),
          ("2021-01-05",322),
          ("2021-01-06",242),
          ("2021-01-07",22),
          ("2021-01-08",46)]

sql_insert2(con,entities)

In [9]:
def sql_fetch_rolsum(con):
  cur=con.cursor()
  cur.execute("""
              SELECT * FROM salesdata 
              GROUP BY date 
              HAVING date=max(date) 
              ORDER BY date
              """)
  rows=cur.fetchall()
  for row in rows:
    print(row)
sql_fetch_rolsum(con)

('2021-01-01', 100)
('2021-01-02', 34)
('2021-01-03', 145)
('2021-01-04', 541)
('2021-01-05', 322)
('2021-01-06', 242)
('2021-01-07', 22)
('2021-01-08', 46)


Answer to question 3

In [10]:
import pandas as pd
import json
import csv

In [11]:
jsondata = [
          {
              "user": {
                  "id": 1,
                  "name": "agus",
                  "email": "agus@example.com"
              },
              "txs": [
                  {
                      "id": 1,
                      "tx_date": "2020-01-01",
                      "tx_amount": 10,
                      "tx_type": "buy"
                  },
                  {
                      "id": 3,
                      "tx_date": "2020-02-01",
                      "tx_amount": 10,
                      "tx_type": "buy"
                  },
                  {
                      "id": 5,
                      "tx_date": "2020-03-01",
                      "tx_amount": 20,
                      "tx_type": "sell"
                  }
              ]
          },
          {
              "user": {
                  "id": 2,
                  "name": "asep",
                  "email": "asep@example.com"
              },
              "txs": [
                  {
                      "id": 2,
                      "tx_date": "2020-01-01",
                      "tx_amount": 10,
                      "tx_type": "sell"
                  },
                  {
                      "id": 4,
                      "tx_date": "2020-02-01",
                      "tx_amount": 10,
                      "tx_type": "sell"
                  },
                  {
                      "id": 6,
                      "tx_date": "2020-03-01",
                      "tx_amount": 20,
                      "tx_type": "buy"
                  }
              ]
          }
       ]

In [12]:
dataframe=pd.json_normalize(jsondata,'txs', [['user','id'], ['user','name'], ['user','email']])
dataframe

,id,tx_date,tx_amount,tx_type,user.id,user.name,user.email
0,1,2020-01-01,10,buy,1,agus,agus@example.com
1,3,2020-02-01,10,buy,1,agus,agus@example.com
2,5,2020-03-01,20,sell,1,agus,agus@example.com
3,2,2020-01-01,10,sell,2,asep,asep@example.com
4,4,2020-02-01,10,sell,2,asep,asep@example.com
5,6,2020-03-01,20,buy,2,asep,asep@example.com


In [13]:
csv1=dataframe[["user.id","id"]]
csv1.to_csv("csv1.csv",header= ['user_id', 'tx_id'],index= False)

In [14]:
csv2 = dataframe[['user.id','user.name', 'user.email']].drop_duplicates().reset_index(drop=True)
csv2.to_csv('csv2.csv', header= ['id', 'name', 'email'], index= False)

In [15]:
csv3 = dataframe[['id', 'tx_date', 'tx_amount', 'tx_type']].sort_values('tx_date')
csv3.to_csv('csv3.csv', index = False)